In [1]:
ls ~

 00-installer-config.yaml             kk_prinseq
 01-network-manager-all.yaml          kkblast
 01-network-manager-all.yaml_resp2    kkipymb
 01-network-manager-all.yaml_resp3    kkmake
 01-network-manager-all.yaml_resp4    kktime
 Anaconda3-2020.07-Linux-x86_64.sh*   major_capsid_all_not_clean.fasta
'Calibre Library'/                    megan/
 Desktop/                             mozilla.pdf
 Documents/                           ncbi/
 Downloads/                           ncbi-outdir/
 FiraxisLive/                         new_drive/
 Games/                               old/
 Music/                               openvpn-install.sh*
 NVIDIA_CUDA-10.1_Samples/            opt/
 PROKKA_10072019.faa                  oscar/
 Pictures/                            oscar.tar
 Public/                              patric-cli-1.032.deb
 Templates/                           perl5/
 Untitled.pdf                         prinseq-plus-plus-1.2/
 VPN/                                 prinseq-plus-plus-1